In [1]:
import sys
sys.path.append('..')
import lcatools
from lcatools.tools import *

In [2]:
from lcatools.exchanges import Exchange

In [8]:
def catalog_name(catalog):
    return [f for f in filter(bool, catalog.split('/'))][-1]

def add_catalog_name(catalog):
    n = catalog_name(catalog.catalog)
    for i in catalog.archive:
        for e in i._entities:
            i[e]['CatalogName'] = n

def add_if_absent(db, entity_list):
    for e in entity_list:
        if e._uuid not in db._entities:
            db[e._uuid] = e


def merge(db1, db2):
    """
    merge contents of db2 into db1
    """
    add_if_absent(db1, db2.quantities())
    db1.check_counter('quantity')
    add_if_absent(db1, db2.flows())
    db1.check_counter('flow')
    add_if_absent(db1, db2.processes())
    db1.check_counter('process')
    print('Before: DB1: %d exchanges' % len(db1.exchanges()))
    print('Before: DB2: %d exchanges' % len(db2.exchanges()))
    for x in db2.exchanges():
        e = Exchange(process=db1[x.process._uuid], 
                    flow=db1[x.flow._uuid],
                    direction=x.direction)
        db1._add_exchange(e)
    print('After:  DB1: %d exchanges' % len(db1.exchanges()))


In [4]:
files = gz_files('/home/b/Dropbox/py-gabi')

In [5]:
files

['/home/b/Dropbox/py-gabi/gabi_2016_extension-database-ia-intermediates-organic.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-ib-intermediates-inorganic.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-ii-energy.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-iii-steel.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-iv-aluminium.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-ix-end-of-life.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-v-non-ferrous-metals.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-vi-precious-metals.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-vii-plastics.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-viii-coatings.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-xi-electronics.json.gz',
 '/home/b/Dropbox/py-gabi/gabi_2016_extension-database-xii-renewable-raw-materials.json.gz',
 '/home/b/

In [6]:
g = from_json(files[0])

Archive refers to a web address using protocol http
Archive refers to a web address using protocol http
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/93a60a57-a4c8-11da-a746-0800200c9a66.xml
1 new quantity entities added (1 total)
71 new flow entities added (71 total)
152 new process entities added (152 total)


In [9]:
add_catalog_name(g)

In [10]:
g_in = from_json(files[1])
add_catalog_name(g_in)
merge(g.archive[0], g_in.archive[0])

Archive refers to a web address using protocol http
Archive refers to a web address using protocol http
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/93a60a57-a4c8-11da-a746-0800200c9a66.xml
1 new quantity entities added (1 total)
22 new flow entities added (22 total)
77 new process entities added (77 total)
0 new quantity entities added (1 total)
22 new flow entities added (93 total)
77 new process entities added (229 total)
Before: DB1: 152 exchanges
Before: DB2: 77 exchanges
After:  DB1: 229 exchanges


In [13]:
g.archive[0].processes()[227]['CatalogName']

'extension-database-ib-intermediates-inorganic'

In [16]:
for i in range(1, len(files)):
    g_in = from_json(files[i])
    add_catalog_name(g_in)
    print('Merging %s...' % g_in.catalog)
    for j in g_in.archive:
        merge(g.archive[0], j)

Merging http://www.gabi-software.com/support/gabi/gabi-database-2016-lci-documentation/extension-database-xiii-ecoinvent-integrated/...
Archive refers to a web address using protocol http
Archive refers to a web address using protocol http
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/93a60a57-a3c8-11da-a746-0800200c9a66.xml
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/3620148f-c5db-48ce-9065-a10092089aca.xml
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/93a60a57-a3c8-18da-a746-0800200c9a66.xml
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/838aaa22-0117-11db-92e3-0800200c9a66.xml
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/93a60a57-a3c8-12da-a746-0800200c9a66.xml
Accessing remote url: http://gabi-documentation-2016.gabi-software.com/xml-data/unitgroups/af638906-

In [18]:
with gzip.open('gabi-2016-all-extension-dbs.json.gz', 'wt') as fp:
    json.dump(g.serialize(exchanges=True), fp, indent=2, sort_keys=True)